<a href="https://colab.research.google.com/github/ayasherby/content/blob/main/Grad_model_Tooth_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Arnold0210/TEETH-RECOGNITION-WITH-MACHINE-LEARNING


Cloning into 'TEETH-RECOGNITION-WITH-MACHINE-LEARNING'...
remote: Enumerating objects: 32505, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 32505 (delta 19), reused 17 (delta 9), pack-reused 32477
Receiving objects: 100% (32505/32505), 6.29 GiB | 38.76 MiB/s, done.
Resolving deltas: 100% (5099/5099), done.
Updating files: 100% (14166/14166), done.


In [84]:
pip install opencv-python-headless numpy tqdm

In [83]:
!pip install numpy pandas matplotlib scikit-learn

In [89]:
import os

# Change the working directory to your project root
project_root = '/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source'
os.chdir(project_root)

# Verify the change
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Source


In [143]:
!python main.py

PreProcessing Directory Already Exists.
FeatureExtraction Directory Already Exists.
Sampling Directory Already Exists.
Classification Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Already Exists.
HSV Directory Already Exists.
LAB Directory Already Exists.
Lab_ Directory Already Exists.
ResizeImages Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Al

In [136]:
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)
            print(list(images_name.index('101_0001.JPG')))

            for train_index, test_index in k_folds.split(images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                for i in train_index:
                    train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:
                    test_features.append(feature.to_numpy()[images_name.index(str(onlyfiles[i].split('.')[0]))])
                    test_label.append(labels_color[images_name.index(str(onlyfiles[i].split('.')[0]))])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()


## **Trying for Batches**

In [3]:
import os
import cv2 as cv
from tqdm import tqdm

class LoadData:
    def __init__(self, path_images):
        self.PATH = path_images

    def read_images_in_batches(self, batch_size=32):
        files = [file for file in os.listdir(self.PATH) if file.lower().endswith(('.jpg', '.jpeg'))]
        total_files = len(files)
        for i in range(0, total_files, batch_size):
            batch_files = files[i:i+batch_size]
            images = []
            names = []
            for filename in batch_files:
                image_path = os.path.join(self.PATH, filename)
                try:
                    img = cv.imread(image_path)
                    if img is not None:
                        images.append(img)
                        names.append(filename)
                    else:
                        print(f"Failed to load image: {filename}")
                except Exception as e:
                    print(f"Error reading {filename}: {e}")
            yield images, names

    def read_one_image(self, image_name):
        image_path = os.path.join(self.PATH, image_name)
        try:
            image = cv.imread(image_path, 1)
            if image is not None:
                return image, image_name
            else:
                print(f"Failed to load image: {image_name}")
        except Exception as e:
            print(f"Error reading {image_name}: {e}")
            return None, image_name

In [21]:
# Copyright (c) 2019. Arnold Julian Herrera Quiñones -  Cristhian Camilo Arce Garcia.
# All Rights Reserved
#
# This product is protected by copyright and distributed under
# licenses restricting copying, distribution, and decompilation.
# It is forbidden the use partial or global of this algorithm  unless authors written permission.

import errno
import os
import sys


import cv2 as cv
import numpy as np
from tqdm import tqdm

# Now try your imports
import Classification as Cl
import FeatureExtraction as fE
import PreProcessingData as pD
import ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Option 1: Read images in batches (using batch processing)
        loader = LoadData("/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/DATASET")
        batch_size = 30  # You can adjust the batch size based on your memory capacity

        for images, names in loader.read_images_in_batches(batch_size):
            # Process each batch here
             print(f"Processed a batch of {len(images)} images")

ModuleNotFoundError: No module named 'Classification'

In [15]:
loader = LoadData("/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/DATASET")
batch_size = 100  # You can adjust the batch size based on your memory capacity

for images, names in loader.read_images_in_batches(batch_size):
    # Process each batch here
    print(f"Processed a batch of {len(images)} images")

Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 100 images
Processed a batch of 73 images


In [121]:
import pandas as pd

# Assuming your features are in 'features.csv' and labels in 'labels.csv'
features_df = pd.read_csv('/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/FeatureExtraction/features.csv')
labels_df = pd.read_csv('/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Labels/Labels.csv')

# Select rows 0 to 937 from features_df (assuming indexing starts from 0)
features_subset = features_df.iloc[0:938]  # Adjust end index to 938 for 937 rows (inclusive)

# Assuming the image name is in the first column of features.csv (adjust if different)
matching_image_names = set(labels_df['Nombre de la imagen'])

# Filter features_subset to keep rows with matching image names in labels_df
features_with_labels = features_subset[features_subset['Imagen'].isin(matching_image_names)]

# Now features_with_labels and labels_df should have a matching number of rows

# Extract labels from labels_df
labels = labels_df['color']  # Adjust column name if needed

# Proceed with training your model using features_with_labels and labels


KeyError: 'Imagen'